In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle

import matplotlib

matplotlib.rcParams['font.family'] ='Malgun Gothic'

matplotlib.rcParams['axes.unicode_minus'] =False

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score, accuracy_score
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# 응, 머신하기전에 종목명부터 바꾸고

In [6]:
edit_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/수정해야할기업54.csv',index_col=0)
edit_df.head(2)

,corp_name,code
9,태양금속공업,4100
41,진양폴리우레탄,10640


In [9]:
code_li = []
for code in edit_df.code:
    code_li.append(f'{code:06}')

In [10]:
edit_df.code = code_li

In [11]:
stock_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/kospi_목록_코드_종목코드.csv',index_col=0)

In [12]:
code_li = []
for code in stock_df.stock_code:
    code_li.append(f'{code:06}')
stock_df.stock_code = code_li

In [20]:
for a in range(len(edit_df)): 
    for b in range(len(stock_df)):
        if stock_df.iloc[b,2] == edit_df.iloc[a,1]:
            stock_df.iloc[b,1] = edit_df.iloc[a,0]

In [26]:
stock_df[stock_df.stock_code == '019490']

,corp_code,corp_name,stock_code
77,156150,하이트론씨스템즈,019490


In [23]:
edit_df.head(5)

,corp_name,code
9,태양금속공업,004100
41,진양폴리우레탄,010640
48,효성 ITX,094280
62,삼영화학공업,003720
77,하이트론씨스템즈,019490


In [27]:
# stock_df.to_csv('c:/Users/bitcamp/Desktop/final_data/kospi_목록_코드_종목코드.csv')

# 진짜 머신러닝

## test

In [3]:
train_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/train/000040.csv',index_col=0)
test_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/test/000040.csv',index_col=0)

In [4]:
test_df

,date,시가,고가,저가,종가,거래량,거래대금,등락률,종목명,시가총액,...,자본총계,매출액,영업이익,법인세차감전 순이익,당기순이익,자본금,수익률,5per,10per,15per
0,2021-09-01,1125,1135,1115,1130,204474,230928240,0.44,40,108635225840,...,5.243179e+10,1.529203e+10,2.234482e+09,7.016995e+08,7.016995e+08,4.806868e+10,-11.0,0,0,0
1,2021-09-02,1145,1145,1120,1130,219167,247221410,0.00,40,108635225840,...,5.243179e+10,1.529203e+10,2.234482e+09,7.016995e+08,7.016995e+08,4.806868e+10,-9.0,0,0,0
2,2021-09-03,1130,1140,1115,1125,308596,347645670,-0.44,40,108154539000,...,5.243179e+10,1.529203e+10,2.234482e+09,7.016995e+08,7.016995e+08,4.806868e+10,-10.0,0,0,0
3,2021-09-06,1130,1130,1105,1120,351201,391703345,-0.44,40,107673852160,...,5.243179e+10,1.529203e+10,2.234482e+09,7.016995e+08,7.016995e+08,4.806868e+10,-10.0,0,0,0
4,2021-09-07,1125,1135,1105,1110,226623,252024010,-0.89,40,106712478480,...,5.243179e+10,1.529203e+10,2.234482e+09,7.016995e+08,7.016995e+08,4.806868e+10,-6.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2022-02-09,808,820,803,817,101104,82399542,1.11,40,78544229656,...,5.022564e+10,9.900577e+10,-2.468118e+09,-1.018985e+10,-1.017489e+10,4.806868e+10,8.0,1,0,0
107,2022-02-10,820,830,814,817,184738,151240356,0.00,40,78544229656,...,5.022564e+10,9.900577e+10,-2.468118e+09,-1.018985e+10,-1.017489e+10,4.806868e+10,10.0,1,0,0
108,2022-02-11,816,816,800,812,98561,79488451,-0.61,40,78063542816,...,5.022564e+10,9.900577e+10,-2.468118e+09,-1.018985e+10,-1.017489e+10,4.806868e+10,9.0,1,0,0
109,2022-02-14,807,807,776,781,128233,100663685,-3.82,40,75083284408,...,5.022564e+10,9.900577e+10,-2.468118e+09,-1.018985e+10,-1.017489e+10,4.806868e+10,12.0,1,1,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1332 entries, 0 to 1331
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1332 non-null   object 
 1   시가          1332 non-null   int64  
 2   고가          1332 non-null   int64  
 3   저가          1332 non-null   int64  
 4   종가          1332 non-null   int64  
 5   거래량         1332 non-null   int64  
 6   거래대금        1332 non-null   int64  
 7   등락률         1332 non-null   float64
 8   종목명         1332 non-null   int64  
 9   시가총액        1332 non-null   int64  
 10  상장주식수       1332 non-null   int64  
 11  수정종가        1332 non-null   int64  
 12  quarter     1332 non-null   object 
 13  유동자산        1332 non-null   float64
 14  비유동자산       1332 non-null   float64
 15  자산총계        1332 non-null   float64
 16  유동부채        1332 non-null   float64
 17  비유동부채       1332 non-null   float64
 18  부채총계        1332 non-null   float64
 19  이익잉여금       1332 non-null  

In [7]:
y = train_df['5per']
x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [12]:
rfc = RandomForestClassifier()
svc = SVC(probability=True)
lr = LogisticRegression()
xg = XGBClassifier()
lbgm = LGBMClassifier()

model = [rfc, svc, lr, xg, lbgm]

for m in model:
    m.fit(x_train, y_train)
    pred = m.predict(x_test)   
    prob = m.predict_proba(x_test)
    print(m)
    print(classification_report(y_test, pred))
    print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('='*100)

RandomForestClassifier()
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       198
           1       0.87      0.75      0.81        69

    accuracy                           0.91       267
   macro avg       0.89      0.86      0.87       267
weighted avg       0.90      0.91      0.90       267

--------------------------------------------------
roc_auc :  0.94737227345923
SVC(probability=True)
              precision    recall  f1-score   support

           0       0.78      0.95      0.86       198
           1       0.65      0.25      0.36        69

    accuracy                           0.77       267
   macro avg       0.72      0.60      0.61       267
weighted avg       0.75      0.77      0.73       267

--------------------------------------------------
roc_auc :  0.8156199677938809
LogisticRegression()
              precision    recall  f1-score   support

           0       0.83      0.95      0.88       198
         

### 문제있는기업

In [ ]:
['동원산업'] : 006040
['일정실업'] : 008500
['호텔신라'] : 008770
['동원시스템즈'] : 014820
['새론오토모티브'] : 075180
['에이블씨엔씨'] : 078520
['바다로19호'] : 155900

In [185]:
yabal_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_통합_data/155900.csv',index_col=0)

In [186]:
yabal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1424 entries, 0 to 1423
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1424 non-null   object 
 1   시가          1424 non-null   int64  
 2   고가          1424 non-null   int64  
 3   저가          1424 non-null   int64  
 4   종가          1424 non-null   int64  
 5   거래량         1424 non-null   int64  
 6   거래대금        1424 non-null   int64  
 7   등락률         1424 non-null   float64
 8   종목명         1424 non-null   int64  
 9   시가총액        1424 non-null   int64  
 10  상장주식수       1424 non-null   int64  
 11  수정종가        1424 non-null   int64  
 12  quarter     1424 non-null   object 
 13  유동자산        1424 non-null   float64
 14  비유동자산       1424 non-null   float64
 15  자산총계        1424 non-null   float64
 16  유동부채        1424 non-null   float64
 17  비유동부채       1424 non-null   object 
 18  부채총계        1424 non-null   float64
 19  자본총계        1424 non-null  

In [188]:
# yabal_df['자산총계'] = yabal_df['자산총계'].apply(lambda x: np.nan if x == '-' else x).astype('float64')
# yabal_df['유동자산'] = yabal_df['유동자산'].apply(lambda x: np.nan if x == '-' else x).astype('float64')
# yabal_df['비유동자산'] = yabal_df['비유동자산'].apply(lambda x: np.nan if x == '-' else x).astype('float64')
# yabal_df['유동부채'] = yabal_df['유동부채'].apply(lambda x: np.nan if x == '-' else x).astype('float64')
yabal_df['비유동부채'] = yabal_df['비유동부채'].apply(lambda x: np.nan if x == '-' else x).astype('float64')
# yabal_df['자본금'] = yabal_df['자본금'].apply(lambda x: np.nan if x == '-' else x).astype('float64')

In [189]:
yabal_df.dropna(inplace=True)
yabal_df.reset_index(drop=True, inplace=True)
yabal_df

,date,시가,고가,저가,종가,거래량,거래대금,등락률,종목명,시가총액,...,부채총계,자본총계,매출액,영업이익,법인세차감전 순이익,당기순이익,수익률,5per,10per,15per
0,2016-05-02,2030,2035,1925,2000,4059,8055010,-0.99,155900,30400008000,...,3.435492e+09,3.737060e+10,2.966331e+09,5.912003e+08,3.290900e+08,3.290900e+08,12.0,1,1,0
1,2016-05-03,1930,2005,1930,2000,34587,68811240,0.00,155900,30400008000,...,3.435492e+09,3.737060e+10,2.966331e+09,5.912003e+08,3.290900e+08,3.290900e+08,12.0,1,1,0
2,2016-05-04,1995,2045,1950,1995,14089,28110295,-0.25,155900,30324007980,...,3.435492e+09,3.737060e+10,2.966331e+09,5.912003e+08,3.290900e+08,3.290900e+08,13.0,1,1,0
3,2016-05-09,2005,2010,1960,1990,9514,19029525,-0.25,155900,30248007960,...,3.435492e+09,3.737060e+10,2.966331e+09,5.912003e+08,3.290900e+08,3.290900e+08,13.0,1,1,0
4,2016-05-10,2030,2035,1970,1990,5214,10373670,0.00,155900,30248007960,...,3.435492e+09,3.737060e+10,2.966331e+09,5.912003e+08,3.290900e+08,3.290900e+08,17.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294,2022-02-09,3065,3080,3060,3080,29414,90266735,0.49,155900,46816012320,...,2.767202e+09,4.465828e+10,7.785143e+09,4.743747e+09,6.393169e+09,6.393169e+09,6.0,1,0,0
1295,2022-02-10,3080,3125,3075,3080,54290,168092410,0.00,155900,46816012320,...,2.767202e+09,4.465828e+10,7.785143e+09,4.743747e+09,6.393169e+09,6.393169e+09,5.0,0,0,0
1296,2022-02-11,3110,3110,3060,3085,14037,43065925,0.16,155900,46892012340,...,2.767202e+09,4.465828e+10,7.785143e+09,4.743747e+09,6.393169e+09,6.393169e+09,1.0,0,0,0
1297,2022-02-14,3095,3100,3030,3080,28454,87073135,-0.16,155900,46816012320,...,2.767202e+09,4.465828e+10,7.785143e+09,4.743747e+09,6.393169e+09,6.393169e+09,2.0,0,0,0


In [190]:
yabal_df.to_csv('c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_통합_data/155900.csv')

## 30일 5per

In [3]:
files = os.listdir('c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/train/')
# files

['000040.csv',
 '000050.csv',
 '000080.csv',
 '000100.csv',
 '000120.csv',
 '000150.csv',
 '000180.csv',
 '000210.csv',
 '000230.csv',
 '000240.csv',
 '000270.csv',
 '000300.csv',
 '000320.csv',
 '000390.csv',
 '000430.csv',
 '000480.csv',
 '000490.csv',
 '000500.csv',
 '000520.csv',
 '000590.csv',
 '000640.csv',
 '000660.csv',
 '000670.csv',
 '000680.csv',
 '000700.csv',
 '000720.csv',
 '000850.csv',
 '000860.csv',
 '000880.csv',
 '000890.csv',
 '000910.csv',
 '000950.csv',
 '000990.csv',
 '001020.csv',
 '001040.csv',
 '001060.csv',
 '001070.csv',
 '001120.csv',
 '001130.csv',
 '001210.csv',
 '001230.csv',
 '001250.csv',
 '001260.csv',
 '001380.csv',
 '001390.csv',
 '001430.csv',
 '001440.csv',
 '001460.csv',
 '001470.csv',
 '001520.csv',
 '001530.csv',
 '001560.csv',
 '001570.csv',
 '001620.csv',
 '001630.csv',
 '001740.csv',
 '001780.csv',
 '001800.csv',
 '001820.csv',
 '001940.csv',
 '002020.csv',
 '002030.csv',
 '002070.csv',
 '002100.csv',
 '002140.csv',
 '002170.csv',
 '002200.c

In [4]:
name_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/kospi_목록_코드_종목코드.csv',index_col=0)
code_li = []
for code in name_df.stock_code:
    code_li.append(f'{code:06}')
name_df.stock_code = code_li
# name_df

,corp_code,corp_name,stock_code
0,960641,한국패러랠,168490
1,600013,맵스리얼티1,094800
2,435297,맥쿼리인프라,088980
3,159698,지역난방공사,071320
4,458234,아시아나IDT,267850
...,...,...,...
815,131780,SK네트웍스,001740
816,217947,신세계건설,034300
817,117212,두산,000150
818,150439,진양산업,003780


In [91]:
# rfc_accuracy = []
# xg_accuracy = []
# lbgm_accuracy = []

for file in files[:5]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/30일/train/{file}',index_col=0)
    test_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/30일/tes{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name)
    
    y = train_df['5per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    rfc = RandomForestClassifier()
#     svc = SVC(probability=True)
#     lr = LogisticRegression()
    xg = XGBClassifier()
    lbgm = LGBMClassifier()

    model = [rfc, xg, lbgm]
    model_name = ['RFC','XG', 'LBGM']
    
    for i,m in enumerate(model):
        m.fit(x_train, y_train)
        pred = m.predict(x_test)   
        prob = m.predict_proba(x_test)
        
#         with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_5per/{file[:6]}_{m.}.pickle','wb') as fw:
#             pickle.dump(m, fw)
        
        print(m)
        print(classification_report(y_test, pred))
        print('-'*50)
        print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
        print('f1_score ; ', f1_score(y_test, pred))
        print('accuracy : ', accuracy_score(y_test, pred))
        print('-'*100)
        
        
    
    print('='*100)

['KR모터스']
RandomForestClassifier()
              precision    recall  f1-score   support

           0       0.91      0.96      0.94       198
           1       0.88      0.74      0.80        69

    accuracy                           0.91       267
   macro avg       0.90      0.85      0.87       267
weighted avg       0.90      0.91      0.90       267

--------------------------------------------------
roc_auc :  0.9457253696384131
f1_score ;  0.8031496062992126
accuracy :  0.9063670411985019
----------------------------------------------------------------------------------------------------
[16:35:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              c

RandomForestClassifier()
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       213
           1       0.78      0.70      0.74        54

    accuracy                           0.90       267
   macro avg       0.85      0.83      0.84       267
weighted avg       0.90      0.90      0.90       267

--------------------------------------------------
roc_auc :  0.9429664406190228
f1_score ;  0.7378640776699029
accuracy :  0.898876404494382
----------------------------------------------------------------------------------------------------
[16:35:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_by

In [200]:
rfc_accuracy = []
rfc_accuracy_f1 = []

for file in files[:]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/train/{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name,':',file[:6])

    y = train_df['5per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    rfc = RandomForestClassifier()

    rfc.fit(x_train, y_train)
    pred = rfc.predict(x_test)   
    prob = rfc.predict_proba(x_test)

#     with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_5per/마지막_test/Randomforest/{file[:6]}.pickle','wb') as fw:
#         pickle.dump(m, fw)


#     print(classification_report(y_test, pred))
#     print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('f1_score ; ', f1_score(y_test, pred))
    print('accuracy : ', accuracy_score(y_test, pred))
    rfc_accuracy.append(roc_auc_score(y_test, prob[:,1]))
    rfc_accuracy_f1.append(f1_score(y_test, pred))
#     print('-'*100)



    print('='*100)
        
#     except:
#         print(stock_name,':',file[:6])

['KR모터스'] : 000040
roc_auc :  0.9419923876445615
f1_score ;  0.7812499999999999
accuracy :  0.8951310861423221
['경방'] : 000050
roc_auc :  0.9618736383442266
f1_score ;  0.7472527472527473
accuracy :  0.9138576779026217
['하이트진로'] : 000080
roc_auc :  0.9597578347578347
f1_score ;  0.8175182481751826
accuracy :  0.9063670411985019
['유한양행'] : 000100
roc_auc :  0.9448356807511737
f1_score ;  0.7547169811320754
accuracy :  0.9026217228464419
['CJ대한통운'] : 000120
roc_auc :  0.9200902583255525
f1_score ;  0.6890756302521008
accuracy :  0.8614232209737828
['두산'] : 000150
roc_auc :  0.9386849414321942
f1_score ;  0.7936507936507936
accuracy :  0.8539325842696629
['성창기업지주'] : 000180
roc_auc :  0.9224864257629657
f1_score ;  0.6170212765957447
accuracy :  0.8651685393258427
['DL'] : 000210
roc_auc :  0.912501989495464
f1_score ;  0.6785714285714286
accuracy :  0.8651685393258427
['일동홀딩스'] : 000230
roc_auc :  0.9452587054888846
f1_score ;  0.6388888888888888
accuracy :  0.9026217228464419
['한국앤컴퍼니']

f1_score ;  0.7874015748031497
accuracy :  0.898876404494382
['동국제강'] : 001230
roc_auc :  0.9577926015767132
f1_score ;  0.8421052631578948
accuracy :  0.8876404494382022
['GS글로벌'] : 001250
roc_auc :  0.9267429193899782
f1_score ;  0.7540983606557377
accuracy :  0.8876404494382022
['남광토건'] : 001260
roc_auc :  0.9755310778914241
f1_score ;  0.8387096774193549
accuracy :  0.9250936329588015
['SG글로벌'] : 001380
roc_auc :  0.9706100188526797
f1_score ;  0.8553459119496856
accuracy :  0.9138576779026217
['KG케미칼'] : 001390
roc_auc :  0.9538364529403955
f1_score ;  0.84
accuracy :  0.9101123595505618
['세아베스틸지주'] : 001430
roc_auc :  0.9505303425491218
f1_score ;  0.7289719626168224
accuracy :  0.8913857677902621
['대한전선'] : 001440
roc_auc :  0.9447115384615383
f1_score ;  0.7716535433070865
accuracy :  0.8913857677902621
['BYC'] : 001460
roc_auc :  0.9720019342359767
f1_score ;  0.6987951807228916
accuracy :  0.9063670411985019
['삼부토건'] : 001470
roc_auc :  0.951927616050354
f1_score ;  0.7438016

roc_auc :  0.9403559403559404
f1_score ;  0.6229508196721312
accuracy :  0.9138576779026217
['미원상사'] : 002840
roc_auc :  0.9551970060725887
f1_score ;  0.8391608391608392
accuracy :  0.9138576779026217
['신풍제지'] : 002870
roc_auc :  0.9630781306605483
f1_score ;  0.8900523560209423
accuracy :  0.9213483146067416
['대유에이텍'] : 002880
roc_auc :  0.9634687208216619
f1_score ;  0.8064516129032259
accuracy :  0.9101123595505618
['유성기업'] : 002920
roc_auc :  0.9789659224441832
f1_score ;  0.746268656716418
accuracy :  0.9363295880149812
['금호건설'] : 002990
roc_auc :  0.9486270753512134
f1_score ;  0.8275862068965517
accuracy :  0.8876404494382022
['부광약품'] : 003000
roc_auc :  0.9442361111111112
f1_score ;  0.8108108108108107
accuracy :  0.8951310861423221
['혜인'] : 003010
roc_auc :  0.9453416149068323
f1_score ;  0.8248587570621468
accuracy :  0.8838951310861424
['세아제강지주'] : 003030
roc_auc :  0.9739090523937297
f1_score ;  0.8456375838926175
accuracy :  0.9138576779026217
['에이프로젠제약'] : 003060
roc_auc

f1_score ;  0.8235294117647058
accuracy :  0.9438202247191011
['남성'] : 004270
roc_auc :  0.9379391100702574
f1_score ;  0.8076923076923077
accuracy :  0.8876404494382022
['세방'] : 004360
roc_auc :  0.9811285029033072
f1_score ;  0.7540983606557378
accuracy :  0.9438202247191011
['농심'] : 004370
roc_auc :  0.9546362634334528
f1_score ;  0.8142857142857143
accuracy :  0.9026217228464419
['송원산업'] : 004430
roc_auc :  0.9429438058748404
f1_score ;  0.8235294117647061
accuracy :  0.8876404494382022
['세방전지'] : 004490
roc_auc :  0.9550989345509893
f1_score ;  0.7692307692307692
accuracy :  0.9213483146067416
['깨끗한나라'] : 004540
roc_auc :  0.8916380789022298
f1_score ;  0.6285714285714286
accuracy :  0.8539325842696629
['현대비앤지스틸'] : 004560
roc_auc :  0.9589362684729065
f1_score ;  0.8358208955223881
accuracy :  0.9176029962546817
['삼천리'] : 004690
roc_auc :  0.9867281105990783
f1_score ;  0.8979591836734694
accuracy :  0.9625468164794008
['한솔테크닉스'] : 004710
roc_auc :  0.962680296877188
f1_score ;  

roc_auc :  0.9368820039551746
f1_score ;  0.7901234567901235
accuracy :  0.8726591760299626
['삼성전자'] : 005930
roc_auc :  0.9702138178222716
f1_score ;  0.8494623655913978
accuracy :  0.8951310861423221
['이수화학'] : 005950
roc_auc :  0.9512175712239377
f1_score ;  0.7850467289719626
accuracy :  0.9138576779026217
['동부건설'] : 005960
roc_auc :  0.9654713998275366
f1_score ;  0.8148148148148148
accuracy :  0.9063670411985019
['동원산업'] : 006040
roc_auc :  0.940418540297475
f1_score ;  0.7796610169491526
accuracy :  0.8980392156862745
['화승인더스트리'] : 006060
roc_auc :  0.9451203208556149
f1_score ;  0.7402597402597403
accuracy :  0.850187265917603
['SK디스커버리'] : 006120
roc_auc :  0.9498341625207297
f1_score ;  0.7716535433070867
accuracy :  0.8913857677902621
['한국전자홀딩스'] : 006200
roc_auc :  0.969880301847515
f1_score ;  0.8481012658227848
accuracy :  0.9101123595505618
['LS'] : 006260
roc_auc :  0.9159959758551308
f1_score ;  0.661764705882353
accuracy :  0.8277153558052435
['녹십자'] : 006280
roc_auc 

...

f1_score ;  0.8823529411764706
accuracy :  0.9250936329588015
['인팩'] : 023810
roc_auc :  0.9345902828136331
f1_score ;  0.7555555555555555
accuracy :  0.8764044943820225
['WISCOM'] : 024070
roc_auc :  0.9487645348837208
f1_score ;  0.8433734939759036
accuracy :  0.951310861423221
['한국콜마홀딩스'] : 024720
roc_auc :  0.9181137095644868
f1_score ;  0.7205882352941176
accuracy :  0.8576779026217228
['대원화성'] : 024890
roc_auc :  0.9370797626895189
f1_score ;  0.8211920529801324
accuracy :  0.898876404494382
['덕양산업'] : 024900
roc_auc :  0.9122569444444444
f1_score ;  0.7428571428571429
accuracy :  0.8651685393258427
['KPX케미칼'] : 025000
roc_auc :  0.9212189156135345
f1_score ;  0.6444444444444444
accuracy :  0.8801498127340824
['에스제이엠홀딩스'] : 025530
roc_auc :  0.9879955947136565
f1_score ;  0.810126582278481
accuracy :  0.9438202247191011
['한국단자공업'] : 025540
roc_auc :  0.9563464837049743
f1_score ;  0.8333333333333334
accuracy :  0.9325842696629213
['미래산업'] : 025560
roc_auc :  0.9602677795166057
f1

roc_auc :  0.9325900514579759
f1_score ;  0.7358490566037736
accuracy :  0.8951310861423221
['SK'] : 034730
roc_auc :  0.9583144881652345
f1_score ;  0.8031496062992125
accuracy :  0.9063670411985019
['지투알'] : 035000
roc_auc :  0.9560759766889704
f1_score ;  0.810126582278481
accuracy :  0.9438202247191011
['백산'] : 035150
roc_auc :  0.9677172582619339
f1_score ;  0.8842105263157894
accuracy :  0.9176029962546817
['강원랜드'] : 035250
roc_auc :  0.9490263995506459
f1_score ;  0.6593406593406593
accuracy :  0.8838951310861424
['NAVER'] : 035420
roc_auc :  0.942899517982944
f1_score ;  0.7872340425531914
accuracy :  0.850187265917603
['카카오'] : 035720
roc_auc :  0.955174030235556
f1_score ;  0.8436018957345972
accuracy :  0.8764044943820225
['콘텐트리중앙'] : 036420
roc_auc :  0.9247515527950311
f1_score ;  0.7558139534883721
accuracy :  0.8426966292134831
['한국가스공사'] : 036460
roc_auc :  0.9413888888888888
f1_score ;  0.7692307692307693
accuracy :  0.8764044943820225
['SNT홀딩스'] : 036530
roc_auc :  0.

f1_score ;  0.8421052631578948
accuracy :  0.9101123595505618
['LG전자'] : 066570
roc_auc :  0.9695609396138238
f1_score ;  0.898876404494382
accuracy :  0.9325842696629213
['세이브존I&C'] : 067830
roc_auc :  0.9920524691358025
f1_score ;  0.847457627118644
accuracy :  0.9662921348314607
['셀트리온'] : 068270
roc_auc :  0.9743177683444513
f1_score ;  0.894736842105263
accuracy :  0.9250936329588015
['삼성출판사'] : 068290
roc_auc :  0.9725872826231249
f1_score ;  0.8535031847133758
accuracy :  0.9138576779026217
['SK렌터카'] : 068400
roc_auc :  0.9570409982174688
f1_score ;  0.8497409326424871
accuracy :  0.8913857677902621
['티케이지휴켐스'] : 069260
roc_auc :  0.9476865671641792
f1_score ;  0.7596899224806203
accuracy :  0.8838951310861424
['대웅제약'] : 069620
roc_auc :  0.9472065285624608
f1_score ;  0.8253968253968254
accuracy :  0.8764044943820225
['한세엠케이'] : 069640
roc_auc :  0.9750478177708224
f1_score ;  0.8148148148148148
accuracy :  0.9250936329588015
['현대백화점'] : 069960
roc_auc :  0.9609375
f1_score ;  

roc_auc :  0.9565053520277402
f1_score ;  0.761904761904762
accuracy :  0.8876404494382022
['KEC'] : 092220
roc_auc :  0.9079725041757676
f1_score ;  0.8
accuracy :  0.8801498127340824
['KPX홀딩스'] : 092230
roc_auc :  0.9822922893006576
f1_score ;  0.8
accuracy :  0.9625468164794008
['LF'] : 093050
roc_auc :  0.9712871287128713
f1_score ;  0.8253968253968254
accuracy :  0.9176029962546817
['후성'] : 093370
roc_auc :  0.9574474299065421
f1_score ;  0.8636363636363636
accuracy :  0.8876404494382022
['효성 ITX'] : 094280
roc_auc :  0.9321257689678741
f1_score ;  0.7919463087248323
accuracy :  0.8838951310861424
['AJ네트웍스'] : 095570
roc_auc :  0.9624319660161954
f1_score ;  0.7682119205298013
accuracy :  0.8689138576779026
['웅진씽크빅'] : 095720
roc_auc :  0.9489532019704432
f1_score ;  0.6666666666666666
accuracy :  0.9250936329588015
['JW홀딩스'] : 096760
roc_auc :  0.9536647992530345
f1_score ;  0.8571428571428571
accuracy :  0.9325842696629213
['SK이노베이션'] : 096770
roc_auc :  0.9592190016103059
f1_sc

roc_auc :  0.9294764059469942
f1_score ;  0.7730061349693251
accuracy :  0.8614232209737828
['바다로19호'] : 155900
roc_auc :  0.9956608811748999
f1_score ;  0.8421052631578947
accuracy :  0.9628099173553719
['애경케미칼'] : 161000
roc_auc :  0.95793875292092
f1_score ;  0.8387096774193549
accuracy :  0.8876404494382022
['한국타이어앤테크놀로지'] : 161390
roc_auc :  0.930311820159536
f1_score ;  0.7244094488188977
accuracy :  0.8689138576779026
['동일고무벨트'] : 163560
roc_auc :  0.9623832365564251
f1_score ;  0.78125
accuracy :  0.947565543071161
['동아에스티'] : 170900
roc_auc :  0.9220085470085471
f1_score ;  0.7131782945736433
accuracy :  0.8614232209737828
['한진칼'] : 180640
roc_auc :  0.9631501249148308
f1_score ;  0.8717948717948718
accuracy :  0.8876404494382022
['아세아시멘트'] : 183190
roc_auc :  0.9292717086834734
f1_score ;  0.7226890756302521
accuracy :  0.8764044943820225
['쿠쿠홀딩스'] : 192400
roc_auc :  0.9192195197044335
f1_score ;  0.6991869918699187
accuracy :  0.8614232209737828
['코스맥스'] : 192820
roc_auc : 

In [201]:
xg_accuracy = []
xg_accuracy_f1 = []

for file in files[:]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/train/{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name,':',file[:6])
    
    y = train_df['5per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    xg = XGBClassifier()

    model = [rfc, xg, lbgm]
    model_name = ['RFC','XG', 'LBGM']
    
    
    xg.fit(x_train, y_train)
    pred = xg.predict(x_test)   
    prob = xg.predict_proba(x_test)

#     with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_5per/XGB/{file[:6]}.pickle','wb') as fw:
#         pickle.dump(m, fw)

#     print(classification_report(y_test, pred))
#     print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('f1_score ; ', f1_score(y_test, pred))
    print('accuracy : ', accuracy_score(y_test, pred))
    xg_accuracy.append(roc_auc_score(y_test, prob[:,1]))
    xg_accuracy_f1.append(f1_score(y_test, pred))
#     print('-'*100)
        
        
    
    print('='*100)

['KR모터스'] : 000040
[22:35:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9507392768262333
f1_score ;  0.7716535433070866
accuracy :  0.8913857677902621
['경방'] : 000050
[22:35:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9316902687000727
f1_score ;  0.711111111111111
accuracy :  0.9026217228464419
['하이트진로'] : 000080
[22:35:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default ev

f1_score ;  0.7999999999999999
accuracy :  0.9026217228464419
['삼일제약'] : 000520
[22:35:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9250475772214902
f1_score ;  0.7424242424242424
accuracy :  0.8726591760299626
['CS홀딩스'] : 000590
[22:35:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9618096357226792
f1_score ;  0.75
accuracy :  0.9325842696629213
['동아쏘시오홀딩스'] : 000640
[22:35:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/lear

roc_auc :  0.9572916666666667
f1_score ;  0.7851851851851853
accuracy :  0.8913857677902621
['CJ'] : 001040
[22:35:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9541232073011734
f1_score ;  0.7692307692307693
accuracy :  0.9101123595505618
['JW중외제약'] : 001060
[22:35:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9269050802139037
f1_score ;  0.7346938775510204
accuracy :  0.8539325842696629
['대한방직'] : 001070
[22:35:17] WARNING: C:/Users/Administrator/workspa

f1_score ;  0.7652173913043478
accuracy :  0.898876404494382
['디아이동일'] : 001530
[22:35:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9563594821020563
f1_score ;  0.8244274809160306
accuracy :  0.9138576779026217
['제일연마'] : 001560
[22:35:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.913878696309087
f1_score ;  0.6315789473684211
accuracy :  0.8951310861423221
['금양'] : 001570
[22:35:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/sr

f1_score ;  0.6304347826086957
accuracy :  0.8726591760299626
['한국수출포장공업'] : 002200
[22:35:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9336846728151076
f1_score ;  0.75
accuracy :  0.8726591760299626
['한일철강'] : 002220
[22:35:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9320884699057289
f1_score ;  0.7272727272727273
accuracy :  0.8539325842696629
['고려제강'] : 002240
[22:35:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learn

...

roc_auc :  0.9269662921348315
f1_score ;  0.8176795580110497
accuracy :  0.8764044943820225
['HSD엔진'] : 082740
[22:36:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9518239722061379
f1_score ;  0.8195121951219512
accuracy :  0.8614232209737828
['대한제강'] : 084010
[22:36:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.929675222629649
f1_score ;  0.7777777777777778
accuracy :  0.8651685393258427
['동양고속'] : 084670
[22:36:02] WARNING: C:/Users/Administrator/workspa

roc_auc :  0.9707112970711297
f1_score ;  0.76
accuracy :  0.9550561797752809
['LF'] : 093050
[22:36:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9506473724295507
f1_score ;  0.8244274809160306
accuracy :  0.9138576779026217
['후성'] : 093370
[22:36:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9560747663551401
f1_score ;  0.8636363636363636
accuracy :  0.8876404494382022
['효성 ITX'] : 094280
[22:36:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64

roc_auc :  0.9392551087673039
f1_score ;  0.7898089171974522
accuracy :  0.8764044943820225
['우진'] : 105840
[22:36:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9319923371647509
f1_score ;  0.7710843373493976
accuracy :  0.8576779026217228
['LX하우시스'] : 108670
[22:36:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9424862313139261
f1_score ;  0.8034188034188033
accuracy :  0.9138576779026217
['영원무역'] : 111770
[22:36:06] WARNING: C:/Users/Administrator/workspa

roc_auc :  0.9587478559176672
f1_score ;  0.8108108108108109
accuracy :  0.9213483146067416
['코오롱플라스틱'] : 138490
[22:36:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9461770623742455
f1_score ;  0.7794117647058824
accuracy :  0.8876404494382022
['이마트'] : 139480
[22:36:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.8848958333333333
f1_score ;  0.6119402985074626
accuracy :  0.8052434456928839
['다이나믹디자인'] : 145210
[22:36:08] WARNING: C:/Users/Administrator/wo

roc_auc :  0.9209224598930481
f1_score ;  0.7612903225806452
accuracy :  0.8614232209737828
['디와이파워'] : 210540
[22:36:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9639520958083831
f1_score ;  0.8743718592964824
accuracy :  0.9063670411985019
['SK디앤디'] : 210980
[22:36:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9606236403190719
f1_score ;  0.8159999999999998
accuracy :  0.9138576779026217
['한솔제지'] : 213500
[22:36:10] WARNING: C:/Users/Administrator/works

In [202]:
lbgm_accuracy = []
lbgm_accuracy_f1 = []

for file in files[:]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/train/{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name,':',file[:6])
    
    y = train_df['5per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    lbgm = LGBMClassifier()

    model = [rfc, xg, lbgm]
    model_name = ['RFC','XG', 'LBGM']
    

    lbgm.fit(x_train, y_train)
    pred = lbgm.predict(x_test)   
    prob = lbgm.predict_proba(x_test)

#     with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_5per/마지막_test/LBGM/{file[:6]}.pickle','wb') as fw:
#         pickle.dump(m, fw)

#     print(classification_report(y_test, pred))
#     print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('f1_score ; ', f1_score(y_test, pred))
    print('accuracy : ', accuracy_score(y_test, pred))
    lbgm_accuracy.append(roc_auc_score(y_test, prob[:,1]))
    lbgm_accuracy_li.append(f1_score(y_test, pred))
#     print('-'*100)
        
        
    
    print('='*100)

['KR모터스'] : 000040
roc_auc :  0.9502269067486458
f1_score ;  0.7656250000000001
accuracy :  0.8876404494382022
['경방'] : 000050
roc_auc :  0.9447167755991285
f1_score ;  0.7311827956989246
accuracy :  0.9063670411985019
['하이트진로'] : 000080
roc_auc :  0.9450142450142449
f1_score ;  0.7338129496402879
accuracy :  0.8614232209737828
['유한양행'] : 000100
roc_auc :  0.9440097374369675
f1_score ;  0.7500000000000001
accuracy :  0.9026217228464419
['CJ대한통운'] : 000120
roc_auc :  0.8953470276999689
f1_score ;  0.6371681415929202
accuracy :  0.846441947565543
['두산'] : 000150
roc_auc :  0.9160125588697017
f1_score ;  0.7772020725388602
accuracy :  0.8389513108614233
['성창기업지주'] : 000180
roc_auc :  0.9255757348811083
f1_score ;  0.6521739130434783
accuracy :  0.8801498127340824
['DL'] : 000210
roc_auc :  0.9017985038994111
f1_score ;  0.6545454545454545
accuracy :  0.8576779026217228
['일동홀딩스'] : 000230
roc_auc :  0.9266181388943536
f1_score ;  0.5866666666666667
accuracy :  0.8838951310861424
['한국앤컴퍼니']

roc_auc :  0.9102085278555868
f1_score ;  0.7058823529411765
accuracy :  0.8689138576779026
['남광토건'] : 001260
roc_auc :  0.9568843430369788
f1_score ;  0.8225806451612904
accuracy :  0.9176029962546817
['SG글로벌'] : 001380
roc_auc :  0.954484244546189
f1_score ;  0.8258064516129032
accuracy :  0.898876404494382
['KG케미칼'] : 001390
roc_auc :  0.951015531660693
f1_score ;  0.7972972972972973
accuracy :  0.8876404494382022
['세아베스틸지주'] : 001430
roc_auc :  0.946096331072857
f1_score ;  0.7142857142857143
accuracy :  0.8801498127340824
['대한전선'] : 001440
roc_auc :  0.9430410691003911
f1_score ;  0.7642276422764227
accuracy :  0.8913857677902621
['BYC'] : 001460
roc_auc :  0.9573500967117988
f1_score ;  0.7126436781609196
accuracy :  0.9063670411985019
['삼부토건'] : 001470
roc_auc :  0.9406766325727773
f1_score ;  0.7580645161290323
accuracy :  0.8876404494382022
['동양'] : 001520
roc_auc :  0.9399201891812001
f1_score ;  0.773109243697479
accuracy :  0.898876404494382
['디아이동일'] : 001530
roc_auc :  0.

roc_auc :  0.9563458519502476
f1_score ;  0.8526315789473684
accuracy :  0.8951310861423221
['대유에이텍'] : 002880
roc_auc :  0.9595782757547464
f1_score ;  0.773109243697479
accuracy :  0.898876404494382
['유성기업'] : 002920
roc_auc :  0.9611045828437133
f1_score ;  0.7647058823529411
accuracy :  0.9400749063670412
['금호건설'] : 002990
roc_auc :  0.9413154533844189
f1_score ;  0.7928994082840236
accuracy :  0.8689138576779026
['부광약품'] : 003000
roc_auc :  0.9384722222222223
f1_score ;  0.8000000000000002
accuracy :  0.8876404494382022
['혜인'] : 003010
roc_auc :  0.9362732919254659
f1_score ;  0.8156424581005586
accuracy :  0.8764044943820225
['세아제강지주'] : 003030
roc_auc :  0.9595396130490044
f1_score ;  0.8
accuracy :  0.8913857677902621
['에이프로젠제약'] : 003060
roc_auc :  0.9348201247842824
f1_score ;  0.7724137931034484
accuracy :  0.8764044943820225
['대웅'] : 003090
roc_auc :  0.9376400791034937
f1_score ;  0.812121212121212
accuracy :  0.8838951310861424
['디아이'] : 003160
roc_auc :  0.92656755009696

f1_score ;  0.7865168539325844
accuracy :  0.8576779026217228
['세방전지'] : 004490
roc_auc :  0.9446347031963469
f1_score ;  0.7777777777777777
accuracy :  0.9250936329588015
['깨끗한나라'] : 004540
roc_auc :  0.8541166380789024
f1_score ;  0.660377358490566
accuracy :  0.8651685393258427
['현대비앤지스틸'] : 004560
roc_auc :  0.9513546798029556
f1_score ;  0.828125
accuracy :  0.9176029962546817
['삼천리'] : 004690
roc_auc :  0.9872811059907834
f1_score ;  0.8936170212765958
accuracy :  0.9625468164794008
['한솔테크닉스'] : 004710
roc_auc :  0.9583041590813612
f1_score ;  0.7692307692307693
accuracy :  0.8764044943820225
['팜젠사이언스'] : 004720
roc_auc :  0.9385245901639344
f1_score ;  0.8072289156626505
accuracy :  0.8801498127340824
['써니전자'] : 004770
roc_auc :  0.9303898659451483
f1_score ;  0.8172043010752688
accuracy :  0.8726591760299626
['효성'] : 004800
roc_auc :  0.9020333961755992
f1_score ;  0.6993865030674846
accuracy :  0.8164794007490637
['덕성'] : 004830
roc_auc :  0.9139791854040858
f1_score ;  0.7402

roc_auc :  0.9311497326203209
f1_score ;  0.7612903225806452
accuracy :  0.8614232209737828
['SK디스커버리'] : 006120
roc_auc :  0.9400723654454997
f1_score ;  0.746031746031746
accuracy :  0.8801498127340824
['한국전자홀딩스'] : 006200
roc_auc :  0.9509497788186313
f1_score ;  0.8
accuracy :  0.8838951310861424
['LS'] : 006260
roc_auc :  0.9045702788157517
f1_score ;  0.6564885496183206
accuracy :  0.8314606741573034
['녹십자'] : 006280
roc_auc :  0.9410019775873434
f1_score ;  0.8160919540229885
accuracy :  0.8801498127340824
['대원전선'] : 006340
roc_auc :  0.935036255767963
f1_score ;  0.8198757763975155
accuracy :  0.8913857677902621
['GS건설'] : 006360
roc_auc :  0.9413343002175489
f1_score ;  0.7972027972027972
accuracy :  0.8913857677902621
['삼성SDI'] : 006400
roc_auc :  0.9340281690140845
f1_score ;  0.8211382113821138
accuracy :  0.8352059925093633
['인스코비'] : 006490
roc_auc :  0.9551161330822345
f1_score ;  0.8852459016393444
accuracy :  0.9213483146067416
['대림통상'] : 006570
roc_auc :  0.9586436170

...

roc_auc :  0.9539879931389365
f1_score ;  0.8181818181818182
accuracy :  0.9250936329588015
['미래산업'] : 025560
roc_auc :  0.9535732916014605
f1_score ;  0.7884615384615384
accuracy :  0.9176029962546817
['제이준코스메틱'] : 025620
roc_auc :  0.9376976142569704
f1_score ;  0.7794117647058824
accuracy :  0.8876404494382022
['한솔홈데코'] : 025750
roc_auc :  0.9126385467980296
f1_score ;  0.6610169491525424
accuracy :  0.850187265917603
['동서'] : 026960
roc_auc :  0.954462959907606
f1_score ;  0.7289719626168225
accuracy :  0.8913857677902621
['BGF'] : 027410
roc_auc :  0.9757671957671958
f1_score ;  0.7804878048780488
accuracy :  0.9325842696629213
['마니커'] : 027740
roc_auc :  0.9538854974426663
f1_score ;  0.7652173913043477
accuracy :  0.898876404494382
['삼성엔지니어링'] : 028050
roc_auc :  0.91922377198302
f1_score ;  0.8041237113402062
accuracy :  0.8576779026217228
['동아지질'] : 028100
roc_auc :  0.9270944741532976
f1_score ;  0.8137254901960784
accuracy :  0.8576779026217228
['삼성물산'] : 028260
roc_auc :  0

roc_auc :  0.9022927689594356
f1_score ;  0.7562189054726368
accuracy :  0.8164794007490637
['팜스코'] : 036580
roc_auc :  0.8999539170506912
f1_score ;  0.6521739130434783
accuracy :  0.8801498127340824
['YG PLUS'] : 037270
roc_auc :  0.9298807281858129
f1_score ;  0.8070175438596491
accuracy :  0.8764044943820225
['하나투어'] : 039130
roc_auc :  0.9097059588862868
f1_score ;  0.6842105263157895
accuracy :  0.8202247191011236
['HDC랩스'] : 039570
roc_auc :  0.9130341880341881
f1_score ;  0.7096774193548387
accuracy :  0.8651685393258427
['상신브레이크'] : 041650
roc_auc :  0.9192456479690522
f1_score ;  0.7126436781609196
accuracy :  0.9063670411985019
['대우조선해양'] : 042660
roc_auc :  0.9722222222222222
f1_score ;  0.8169014084507044
accuracy :  0.9026217228464419
['현대두산인프라코어'] : 042670
roc_auc :  0.9613975155279503
f1_score ;  0.8478260869565218
accuracy :  0.8951310861423221
['한미반도체'] : 042700
roc_auc :  0.931221198156682
f1_score ;  0.8018433179723502
accuracy :  0.8389513108614233
['KSS해운'] : 0444

roc_auc :  0.9122994652406418
f1_score ;  0.7162162162162162
accuracy :  0.8426966292134831
['농심홀딩스'] : 072710
roc_auc :  0.9401353965183752
f1_score ;  0.7032967032967034
accuracy :  0.898876404494382
['금호타이어'] : 073240
roc_auc :  0.921219715956558
f1_score ;  0.7706422018348624
accuracy :  0.9063670411985019
['이엔플러스'] : 074610
roc_auc :  0.975949975949976
f1_score ;  0.8855721393034826
accuracy :  0.9138576779026217
['새론오토모티브'] : 075180
roc_auc :  0.9870907639073729
f1_score ;  0.8695652173913043
accuracy :  0.9647058823529412
['세진중공업'] : 075580
roc_auc :  0.9384143463898065
f1_score ;  0.825242718446602
accuracy :  0.8651685393258427
['유니퀘스트'] : 077500
roc_auc :  0.9194011976047904
f1_score ;  0.7628865979381443
accuracy :  0.8277153558052435
['텔코웨어'] : 078000
roc_auc :  0.9396050079351085
f1_score ;  0.72
accuracy :  0.8951310861423221
['에이블씨엔씨'] : 078520
roc_auc :  0.9651982004923182
f1_score ;  0.8391608391608392
accuracy :  0.9
['CJ CGV'] : 079160
roc_auc :  0.9100650310375408
f

roc_auc :  0.9458061435619927
f1_score ;  0.7580645161290323
accuracy :  0.8876404494382022
['진양홀딩스'] : 100250
roc_auc :  0.9840188014101057
f1_score ;  0.7999999999999999
accuracy :  0.951310861423221
['동성케미컬'] : 102260
roc_auc :  0.932258064516129
f1_score ;  0.8
accuracy :  0.9288389513108615
['쌍방울'] : 102280
roc_auc :  0.8989440686355388
f1_score ;  0.6734693877551021
accuracy :  0.8801498127340824
['일진전기'] : 103590
roc_auc :  0.9202208682406702
f1_score ;  0.7154471544715447
accuracy :  0.8689138576779026
['한세실업'] : 105630
roc_auc :  0.9436058009228742
f1_score ;  0.7924528301886793
accuracy :  0.8764044943820225
['우진'] : 105840
roc_auc :  0.9385696040868455
f1_score ;  0.761904761904762
accuracy :  0.850187265917603
['LX하우시스'] : 108670
roc_auc :  0.9339889850511408
f1_score ;  0.8032786885245902
accuracy :  0.9101123595505618
['영원무역'] : 111770
roc_auc :  0.9346873755475906
f1_score ;  0.8129032258064517
accuracy :  0.8913857677902621
['씨에스윈드'] : 112610
roc_auc :  0.94024872809496

roc_auc :  0.9548948513415518
f1_score ;  0.8064516129032259
accuracy :  0.9101123595505618
['한솔제지'] : 213500
roc_auc :  0.9354202401372214
f1_score ;  0.7216494845360825
accuracy :  0.898876404494382
['이노션'] : 214320
roc_auc :  0.9007229765473461
f1_score ;  0.6542056074766356
accuracy :  0.8614232209737828
['금호에이치티'] : 214330
roc_auc :  0.9481933674311169
f1_score ;  0.8380952380952381
accuracy :  0.9363295880149812
['토니모리'] : 214420
roc_auc :  0.9534702769996888
f1_score ;  0.787878787878788
accuracy :  0.8951310861423221
['잇츠한불'] : 226320
roc_auc :  0.9540417209908736
f1_score ;  0.7964601769911505
accuracy :  0.9138576779026217
['엘브이엠씨'] : 900140
roc_auc :  0.9578537576167908
f1_score ;  0.7927927927927927
accuracy :  0.9138576779026217


In [203]:
sum(rfc_accuracy)/len(rfc_accuracy)

0.947522324981129

In [204]:
sum(xg_accuracy)/len(xg_accuracy)

0.936343853270578

In [205]:
sum(lbgm_accuracy)/len(lbgm_accuracy)

0.9379878229724077

## 30일 10per

In [207]:
rfc_accuracy_10 = []
rfc_accuracy_f1_10 = []

for file in files[:]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/30일/train/{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name,':',file[:6])

    y = train_df['10per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    rfc = RandomForestClassifier()

    rfc.fit(x_train, y_train)
    pred = rfc.predict(x_test)   
    prob = rfc.predict_proba(x_test)

    with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_10per/마지막_test/Randomforest/{file[:6]}.pickle','wb') as fw:
        pickle.dump(m, fw)


#     print(classification_report(y_test, pred))
#     print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('f1_score ; ', f1_score(y_test, pred))
    print('accuracy : ', accuracy_score(y_test, pred))
    rfc_accuracy_10.append(roc_auc_score(y_test, prob[:,1]))
    rfc_accuracy_f1_10.append(f1_score(y_test, pred))
#     print('-'*100)



    print('='*100)
        
#     except:
#         print(stock_name,':',file[:6])

['KR모터스'] : 000040
roc_auc :  0.9730308219178081
f1_score ;  0.8387096774193549
accuracy :  0.9438202247191011
['경방'] : 000050
roc_auc :  0.9550754458161865
f1_score ;  0.5945945945945945
accuracy :  0.9438202247191011
['하이트진로'] : 000080
roc_auc :  0.951135852451642
f1_score ;  0.736842105263158
accuracy :  0.9250936329588015
['유한양행'] : 000100
roc_auc :  0.9414440572977159
f1_score ;  0.7027027027027026
accuracy :  0.9588014981273408
['CJ대한통운'] : 000120
roc_auc :  0.9416835699797161
f1_score ;  0.6530612244897959
accuracy :  0.9363295880149812
['두산'] : 000150
roc_auc :  0.9460317460317461
f1_score ;  0.6857142857142857
accuracy :  0.8764044943820225
['성창기업지주'] : 000180
roc_auc :  0.9589849984040856
f1_score ;  0.6808510638297872
accuracy :  0.9438202247191011
['DL'] : 000210
roc_auc :  0.9216820987654321
f1_score ;  0.6909090909090909
accuracy :  0.9363295880149812
['일동홀딩스'] : 000230
roc_auc :  0.9872241579558653
f1_score ;  0.6666666666666666
accuracy :  0.9550561797752809
['한국앤컴퍼니'] 

roc_auc :  0.9546527641156798
f1_score ;  0.7654320987654321
accuracy :  0.9288389513108615
['동국제강'] : 001230
roc_auc :  0.9535786145444093
f1_score ;  0.7244094488188976
accuracy :  0.8689138576779026
['GS글로벌'] : 001250
roc_auc :  0.9621195769479818
f1_score ;  0.7894736842105263
accuracy :  0.9400749063670412
['남광토건'] : 001260
roc_auc :  0.9799323017408124
f1_score ;  0.7999999999999999
accuracy :  0.9288389513108615
['SG글로벌'] : 001380
roc_auc :  0.9678129074315515
f1_score ;  0.8095238095238095
accuracy :  0.9101123595505618
['KG케미칼'] : 001390
roc_auc :  0.9606465809072444
f1_score ;  0.7676767676767676
accuracy :  0.9138576779026217
['세아베스틸지주'] : 001430
roc_auc :  0.9517241379310346
f1_score ;  0.7058823529411764
accuracy :  0.9250936329588015
['대한전선'] : 001440
roc_auc :  0.9628571428571429
f1_score ;  0.8
accuracy :  0.9363295880149812
['BYC'] : 001460
roc_auc :  0.9506072874493927
f1_score ;  0.6857142857142857
accuracy :  0.9588014981273408
['삼부토건'] : 001470
roc_auc :  0.9634121

['SUN&L'] : 002820
roc_auc :  0.9843117408906882
f1_score ;  0.8000000000000002
accuracy :  0.9700374531835206
['미원상사'] : 002840
roc_auc :  0.9454749286587852
f1_score ;  0.7857142857142856
accuracy :  0.9325842696629213
['신풍제지'] : 002870
roc_auc :  0.974679487179487
f1_score ;  0.8489208633093526
accuracy :  0.9213483146067416
['대유에이텍'] : 002880
roc_auc :  0.9758098006644518
f1_score ;  0.825
accuracy :  0.947565543071161
['유성기업'] : 002920
roc_auc :  0.9897908366533864
f1_score ;  0.7586206896551724
accuracy :  0.9737827715355806
['금호건설'] : 002990
roc_auc :  0.9386382623224728
f1_score ;  0.712871287128713
accuracy :  0.8913857677902621
['부광약품'] : 003000
roc_auc :  0.949806576402321
f1_score ;  0.8421052631578947
accuracy :  0.9438202247191011
['혜인'] : 003010
roc_auc :  0.924120643004934
f1_score ;  0.7454545454545455
accuracy :  0.8951310861423221
['세아제강지주'] : 003030
roc_auc :  0.9578341013824884
f1_score ;  0.8484848484848485
accuracy :  0.9438202247191011
['에이프로젠제약'] : 003060
roc_a

roc_auc :  0.9483133368109893
f1_score ;  0.7766990291262137
accuracy :  0.9138576779026217
['세방'] : 004360
roc_auc :  0.9997176736307171
f1_score ;  0.9285714285714286
accuracy :  0.9925093632958801
['농심'] : 004370
roc_auc :  0.967558697298662
f1_score ;  0.6206896551724139
accuracy :  0.9176029962546817
['송원산업'] : 004430
roc_auc :  0.9230735320049374
f1_score ;  0.7238095238095238
accuracy :  0.8913857677902621
['세방전지'] : 004490
roc_auc :  0.9777260638297873
f1_score ;  0.7666666666666666
accuracy :  0.947565543071161
['깨끗한나라'] : 004540
roc_auc :  0.9397167487684729
f1_score ;  0.5833333333333333
accuracy :  0.8876404494382022
['현대비앤지스틸'] : 004560
roc_auc :  0.9675132275132274
f1_score ;  0.7792207792207793
accuracy :  0.9363295880149812
['삼천리'] : 004690
roc_auc :  0.9774487030584591
f1_score ;  0.6842105263157895
accuracy :  0.9550561797752809
['한솔테크닉스'] : 004710
roc_auc :  0.9735735735735737
f1_score ;  0.8
accuracy :  0.9400749063670412
['팜젠사이언스'] : 004720
roc_auc :  0.95835217850

f1_score ;  0.7207207207207207
accuracy :  0.8838951310861424
['삼성전자'] : 005930
roc_auc :  0.9047017797017798
f1_score ;  0.5666666666666667
accuracy :  0.9026217228464419
['이수화학'] : 005950
roc_auc :  0.9583128078817733
f1_score ;  0.7761194029850748
accuracy :  0.9438202247191011
['동부건설'] : 005960
roc_auc :  0.9562000726216413
f1_score ;  0.8387096774193549
accuracy :  0.9438202247191011
['동원산업'] : 006040
roc_auc :  0.9491693528390777
f1_score ;  0.6666666666666667
accuracy :  0.9137254901960784
['화승인더스트리'] : 006060
roc_auc :  0.9752073732718894
f1_score ;  0.8125
accuracy :  0.9325842696629213
['SK디스커버리'] : 006120
roc_auc :  0.9653255951209917
f1_score ;  0.775
accuracy :  0.9325842696629213
['한국전자홀딩스'] : 006200
roc_auc :  0.9657534246575342
f1_score ;  0.7816091954022988
accuracy :  0.9288389513108615
['LS'] : 006260
roc_auc :  0.9436358447488584
f1_score ;  0.6888888888888889
accuracy :  0.8951310861423221
['녹십자'] : 006280
roc_auc :  0.9436607300299771
f1_score ;  0.702127659574468

...

f1_score ;  0.6666666666666665
accuracy :  0.9213483146067416
['인지컨트롤스'] : 023800
roc_auc :  0.983320895522388
f1_score ;  0.84375
accuracy :  0.9250936329588015
['인팩'] : 023810
roc_auc :  0.9526764515432764
f1_score ;  0.6666666666666667
accuracy :  0.898876404494382
['WISCOM'] : 024070
roc_auc :  0.9930278884462151
f1_score ;  0.7999999999999999
accuracy :  0.9775280898876404
['한국콜마홀딩스'] : 024720
roc_auc :  0.9258805374001453
f1_score ;  0.7551020408163265
accuracy :  0.9101123595505618
['대원화성'] : 024890
roc_auc :  0.9627890801599721
f1_score ;  0.7878787878787877
accuracy :  0.9213483146067416
['덕양산업'] : 024900
roc_auc :  0.9644216133942161
f1_score ;  0.7906976744186046
accuracy :  0.9325842696629213
['KPX케미칼'] : 025000
roc_auc :  0.9773519163763066
f1_score ;  0.7619047619047619
accuracy :  0.9625468164794008
['에스제이엠홀딩스'] : 025530
roc_auc :  0.9846862549800797
f1_score ;  0.7272727272727272
accuracy :  0.9662921348314607
['한국단자공업'] : 025540
roc_auc :  0.9802631578947368
f1_score ;

f1_score ;  0.7126436781609196
accuracy :  0.9063670411985019
['NICE'] : 034310
roc_auc :  0.9707112970711296
f1_score ;  0.7636363636363638
accuracy :  0.951310861423221
['SK'] : 034730
roc_auc :  0.9003078817733989
f1_score ;  0.5846153846153846
accuracy :  0.898876404494382
['지투알'] : 035000
roc_auc :  0.984
f1_score ;  0.6842105263157895
accuracy :  0.9550561797752809
['백산'] : 035150
roc_auc :  0.9632142318709483
f1_score ;  0.8682170542635659
accuracy :  0.9363295880149812
['강원랜드'] : 035250
roc_auc :  0.9830588235294118
f1_score ;  0.6875
accuracy :  0.9625468164794008
['NAVER'] : 035420
roc_auc :  0.9264679950625991
f1_score ;  0.74
accuracy :  0.9026217228464419
['카카오'] : 035720
roc_auc :  0.9709784411276948
f1_score ;  0.859375
accuracy :  0.9325842696629213
['콘텐트리중앙'] : 036420
roc_auc :  0.9730638486550399
f1_score ;  0.8444444444444444
accuracy :  0.9213483146067416
['한국가스공사'] : 036460
roc_auc :  0.9365063187933143
f1_score ;  0.674698795180723
accuracy :  0.898876404494382
['

f1_score ;  0.7499999999999999
accuracy :  0.8951310861423221
['LG전자'] : 066570
roc_auc :  0.9514555468135328
f1_score ;  0.8130081300813008
accuracy :  0.9138576779026217
['세이브존I&C'] : 067830
roc_auc :  0.9811468094600624
f1_score ;  0.6666666666666666
accuracy :  0.9550561797752809
['셀트리온'] : 068270
roc_auc :  0.9592844139476264
f1_score ;  0.8296296296296297
accuracy :  0.9138576779026217
['삼성출판사'] : 068290
roc_auc :  0.958133971291866
f1_score ;  0.7627118644067795
accuracy :  0.8951310861423221
['SK렌터카'] : 068400
roc_auc :  0.9465823650034176
f1_score ;  0.7887323943661972
accuracy :  0.8876404494382022
['티케이지휴켐스'] : 069260
roc_auc :  0.9607142857142857
f1_score ;  0.6785714285714285
accuracy :  0.9325842696629213
['대웅제약'] : 069620
roc_auc :  0.9567663257277734
f1_score ;  0.7868852459016393
accuracy :  0.9026217228464419
['한세엠케이'] : 069640
roc_auc :  0.9575949367088608
f1_score ;  0.7636363636363636
accuracy :  0.951310861423221
['현대백화점'] : 069960
roc_auc :  0.9053347280334729
f1

f1_score ;  0.8219178082191781
accuracy :  0.951310861423221
['KEC'] : 092220
roc_auc :  0.9616123642439431
f1_score ;  0.8037383177570094
accuracy :  0.9213483146067416
['KPX홀딩스'] : 092230
roc_auc :  0.989010989010989
f1_score ;  0.7142857142857143
accuracy :  0.9850187265917603
['LF'] : 093050
roc_auc :  0.9734592892487629
f1_score ;  0.7951807228915662
accuracy :  0.9363295880149812
['후성'] : 093370
roc_auc :  0.9458689458689459
f1_score ;  0.7482993197278911
accuracy :  0.8614232209737828
['효성 ITX'] : 094280
roc_auc :  0.9463057661788046
f1_score ;  0.7843137254901962
accuracy :  0.9176029962546817
['AJ네트웍스'] : 095570
roc_auc :  0.9418362023995828
f1_score ;  0.6391752577319588
accuracy :  0.8689138576779026
['웅진씽크빅'] : 095720
roc_auc :  0.9448235294117647
f1_score ;  0.7500000000000001
accuracy :  0.9700374531835206
['JW홀딩스'] : 096760
roc_auc :  0.9607929515418502
f1_score ;  0.775
accuracy :  0.9325842696629213
['SK이노베이션'] : 096770
roc_auc :  0.9508331229487503
f1_score ;  0.76470

roc_auc :  0.9423484354628422
f1_score ;  0.784
accuracy :  0.898876404494382
['한국타이어앤테크놀로지'] : 161390
roc_auc :  0.9466755319148935
f1_score ;  0.6666666666666666
accuracy :  0.9250936329588015
['동일고무벨트'] : 163560
roc_auc :  0.9464641434262949
f1_score ;  0.7857142857142857
accuracy :  0.9775280898876404
['동아에스티'] : 170900
roc_auc :  0.9462728405315615
f1_score ;  0.6588235294117646
accuracy :  0.8913857677902621
['한진칼'] : 180640
roc_auc :  0.9832480506822612
f1_score ;  0.9166666666666666
accuracy :  0.9400749063670412
['아세아시멘트'] : 183190
roc_auc :  0.9806167400881058
f1_score ;  0.7901234567901235
accuracy :  0.9363295880149812
['쿠쿠홀딩스'] : 192400
roc_auc :  0.9438904317091643
f1_score ;  0.6956521739130436
accuracy :  0.9213483146067416
['코스맥스'] : 192820
roc_auc :  0.8676497695852534
f1_score ;  0.5977011494252874
accuracy :  0.8689138576779026
['서연이화'] : 200880
roc_auc :  0.9453283683141503
f1_score ;  0.7735849056603773
accuracy :  0.9101123595505618
['만도'] : 204320
roc_auc :  0.9

In [209]:
lbgm_accuracy_10 = []
lbgm_accuracy_f1_10 = []

for file in files[:]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/30일/train/{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name,':',file[:6])
    
    y = train_df['10per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    lbgm = LGBMClassifier()

    model = [rfc, xg, lbgm]
    model_name = ['RFC','XG', 'LBGM']
    

    lbgm.fit(x_train, y_train)
    pred = lbgm.predict(x_test)   
    prob = lbgm.predict_proba(x_test)

    with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_10per/마지막_test/LBGM/{file[:6]}.pickle','wb') as fw:
        pickle.dump(m, fw)

#     print(classification_report(y_test, pred))
#     print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('f1_score ; ', f1_score(y_test, pred))
    print('accuracy : ', accuracy_score(y_test, pred))
    lbgm_accuracy_10.append(roc_auc_score(y_test, prob[:,1]))
    lbgm_accuracy_f1_10.append(f1_score(y_test, pred))
#     print('-'*100)
        
        
    
    print('='*100)

['KR모터스'] : 000040
roc_auc :  0.9684170471841704
f1_score ;  0.787878787878788
accuracy :  0.9213483146067416
['경방'] : 000050
roc_auc :  0.9298696844993141
f1_score ;  0.5853658536585366
accuracy :  0.9363295880149812
['하이트진로'] : 000080
roc_auc :  0.9456815114709852
f1_score ;  0.6666666666666666
accuracy :  0.9026217228464419
['유한양행'] : 000100
roc_auc :  0.9589624467673248
f1_score ;  0.6666666666666666
accuracy :  0.951310861423221
['CJ대한통운'] : 000120
roc_auc :  0.920747609388583
f1_score ;  0.679245283018868
accuracy :  0.9363295880149812
['두산'] : 000150
roc_auc :  0.9091896407685881
f1_score ;  0.6727272727272728
accuracy :  0.8651685393258427
['성창기업지주'] : 000180
roc_auc :  0.9396744334503672
f1_score ;  0.6666666666666666
accuracy :  0.9363295880149812
['DL'] : 000210
roc_auc :  0.9175925925925925
f1_score ;  0.6923076923076923
accuracy :  0.9400749063670412
['일동홀딩스'] : 000230
roc_auc :  0.9690282617111886
f1_score ;  0.5454545454545454
accuracy :  0.9438202247191011
['한국앤컴퍼니'] : 

f1_score ;  0.7777777777777778
accuracy :  0.8951310861423221
['GS글로벌'] : 001250
roc_auc :  0.9475501834664364
f1_score ;  0.7466666666666667
accuracy :  0.9288389513108615
['남광토건'] : 001260
roc_auc :  0.9695357833655706
f1_score ;  0.8125
accuracy :  0.9325842696629213
['SG글로벌'] : 001380
roc_auc :  0.9652868318122555
f1_score ;  0.8
accuracy :  0.9101123595505618
['KG케미칼'] : 001390
roc_auc :  0.9499830737982397
f1_score ;  0.7524752475247526
accuracy :  0.9063670411985019
['세아베스틸지주'] : 001430
roc_auc :  0.9417487684729063
f1_score ;  0.6969696969696969
accuracy :  0.9250936329588015
['대한전선'] : 001440
roc_auc :  0.962962962962963
f1_score ;  0.7954545454545455
accuracy :  0.9325842696629213
['BYC'] : 001460
roc_auc :  0.9647773279352226
f1_score ;  0.5714285714285715
accuracy :  0.9438202247191011
['삼부토건'] : 001470
roc_auc :  0.9548512026090501
f1_score ;  0.7096774193548386
accuracy :  0.898876404494382
['동양'] : 001520
roc_auc :  0.9635715927373019
f1_score ;  0.7575757575757577
accur

roc_auc :  0.9732193732193732
f1_score ;  0.8633093525179857
accuracy :  0.9288389513108615
['대유에이텍'] : 002880
roc_auc :  0.9724356312292358
f1_score ;  0.759493670886076
accuracy :  0.9288389513108615
['유성기업'] : 002920
roc_auc :  0.9900398406374502
f1_score ;  0.7333333333333334
accuracy :  0.9700374531835206
['금호건설'] : 002990
roc_auc :  0.9379281537176274
f1_score ;  0.7407407407407408
accuracy :  0.8951310861423221
['부광약품'] : 003000
roc_auc :  0.9664893617021277
f1_score ;  0.8131868131868133
accuracy :  0.9363295880149812
['혜인'] : 003010
roc_auc :  0.9339487505968487
f1_score ;  0.7610619469026548
accuracy :  0.898876404494382
['세아제강지주'] : 003030
roc_auc :  0.9552995391705069
f1_score ;  0.8484848484848485
accuracy :  0.9438202247191011
['에이프로젠제약'] : 003060
roc_auc :  0.958904109589041
f1_score ;  0.7333333333333334
accuracy :  0.9101123595505618
['대웅'] : 003090
roc_auc :  0.9156403940886699
f1_score ;  0.734375
accuracy :  0.8726591760299626
['디아이'] : 003160
roc_auc :  0.932368621

f1_score ;  0.6774193548387097
accuracy :  0.9250936329588015
['송원산업'] : 004430
roc_auc :  0.9240433785928407
f1_score ;  0.6909090909090909
accuracy :  0.8726591760299626
['세방전지'] : 004490
roc_auc :  0.973404255319149
f1_score ;  0.6885245901639345
accuracy :  0.9288389513108615
['깨끗한나라'] : 004540
roc_auc :  0.9405172413793103
f1_score ;  0.6470588235294118
accuracy :  0.9101123595505618
['현대비앤지스틸'] : 004560
roc_auc :  0.9571428571428571
f1_score ;  0.7272727272727272
accuracy :  0.9213483146067416
['삼천리'] : 004690
roc_auc :  0.9525745257452574
f1_score ;  0.6500000000000001
accuracy :  0.947565543071161
['한솔테크닉스'] : 004710
roc_auc :  0.9648648648648649
f1_score ;  0.7179487179487178
accuracy :  0.9176029962546817
['팜젠사이언스'] : 004720
roc_auc :  0.9585406301824212
f1_score ;  0.7846153846153846
accuracy :  0.8951310861423221
['써니전자'] : 004770
roc_auc :  0.9577790225458619
f1_score ;  0.7826086956521738
accuracy :  0.8876404494382022
['효성'] : 004800
roc_auc :  0.93781146179402
f1_score 

f1_score ;  0.6461538461538462
accuracy :  0.9098039215686274
['화승인더스트리'] : 006060
roc_auc :  0.9476497695852534
f1_score ;  0.8041237113402062
accuracy :  0.9288389513108615
['SK디스커버리'] : 006120
roc_auc :  0.9649813102498526
f1_score ;  0.75
accuracy :  0.9250936329588015
['한국전자홀딩스'] : 006200
roc_auc :  0.9356925418569254
f1_score ;  0.7126436781609194
accuracy :  0.9063670411985019
['LS'] : 006260
roc_auc :  0.9231354642313545
f1_score ;  0.6818181818181818
accuracy :  0.8951310861423221
['녹십자'] : 006280
roc_auc :  0.9258508199612061
f1_score ;  0.7422680412371134
accuracy :  0.9063670411985019
['대원전선'] : 006340
roc_auc :  0.9538761002031144
f1_score ;  0.7857142857142857
accuracy :  0.9101123595505618
['GS건설'] : 006360
roc_auc :  0.9639639639639639
f1_score ;  0.7865168539325843
accuracy :  0.9288389513108615
['삼성SDI'] : 006400
roc_auc :  0.9052769357949848
f1_score ;  0.6883116883116882
accuracy :  0.8202247191011236
['인스코비'] : 006490
roc_auc :  0.9792179802955666
f1_score ;  0.835

...

roc_auc :  0.964769647696477
f1_score ;  0.6956521739130435
accuracy :  0.947565543071161
['에스제이엠홀딩스'] : 025530
roc_auc :  0.9831922310756972
f1_score ;  0.6666666666666667
accuracy :  0.9588014981273408
['한국단자공업'] : 025540
roc_auc :  0.970872694556905
f1_score ;  0.8051948051948051
accuracy :  0.9438202247191011
['미래산업'] : 025560
roc_auc :  0.9584700833122948
f1_score ;  0.7164179104477613
accuracy :  0.9288389513108615
['제이준코스메틱'] : 025620
roc_auc :  0.9506172839506172
f1_score ;  0.7234042553191489
accuracy :  0.9026217228464419
['한솔홈데코'] : 025750
roc_auc :  0.9454759106933021
f1_score ;  0.6956521739130435
accuracy :  0.9213483146067416
['동서'] : 026960
roc_auc :  0.9855112083105522
f1_score ;  0.7931034482758621
accuracy :  0.9550561797752809
['BGF'] : 027410
roc_auc :  0.9787037037037037
f1_score ;  0.72
accuracy :  0.947565543071161
['마니커'] : 027740
roc_auc :  0.9762123649735692
f1_score ;  0.7945205479452055
accuracy :  0.9438202247191011
['삼성엔지니어링'] : 028050
roc_auc :  0.887962

roc_auc :  0.9360986547085203
f1_score ;  0.617283950617284
accuracy :  0.8838951310861424
['SNT홀딩스'] : 036530
roc_auc :  0.9484026660537808
f1_score ;  0.6666666666666667
accuracy :  0.9213483146067416
['엔씨소프트'] : 036570
roc_auc :  0.9224096769059367
f1_score ;  0.6782608695652174
accuracy :  0.8614232209737828
['팜스코'] : 036580
roc_auc :  0.8551255230125523
f1_score ;  0.5581395348837209
accuracy :  0.9288389513108615
['YG PLUS'] : 037270
roc_auc :  0.9605379840378362
f1_score ;  0.806451612903226
accuracy :  0.9101123595505618
['하나투어'] : 039130
roc_auc :  0.8724210897548933
f1_score ;  0.5454545454545454
accuracy :  0.850187265917603
['HDC랩스'] : 039570
roc_auc :  0.9149001222992255
f1_score ;  0.7692307692307693
accuracy :  0.9325842696629213
['상신브레이크'] : 041650
roc_auc :  0.9698823529411764
f1_score ;  0.625
accuracy :  0.9550561797752809
['대우조선해양'] : 042660
roc_auc :  0.9531828601657556
f1_score ;  0.7777777777777778
accuracy :  0.9101123595505618
['현대두산인프라코어'] : 042670
roc_auc :  

roc_auc :  0.934997011356844
f1_score ;  0.5333333333333333
accuracy :  0.9213483146067416
['코아스'] : 071950
roc_auc :  0.9108794019279953
f1_score ;  0.7216494845360826
accuracy :  0.898876404494382
['STX중공업'] : 071970
roc_auc :  0.9678170263080107
f1_score ;  0.8175182481751825
accuracy :  0.9063670411985019
['유엔젤'] : 072130
roc_auc :  0.9252303947139627
f1_score ;  0.6595744680851063
accuracy :  0.8801498127340824
['농심홀딩스'] : 072710
roc_auc :  0.9422132976349844
f1_score ;  0.42857142857142855
accuracy :  0.9400749063670412
['금호타이어'] : 073240
roc_auc :  0.9702367271891519
f1_score ;  0.868421052631579
accuracy :  0.9625468164794008
['이엔플러스'] : 074610
roc_auc :  0.9816743572841133
f1_score ;  0.8837209302325582
accuracy :  0.9250936329588015
['새론오토모티브'] : 075180
roc_auc :  0.9871304347826086
f1_score ;  0.8363636363636363
accuracy :  0.9647058823529412
['세진중공업'] : 075580
roc_auc :  0.9518539810290314
f1_score ;  0.8175182481751825
accuracy :  0.9063670411985019
['유니퀘스트'] : 077500
roc_

roc_auc :  0.9695906432748538
f1_score ;  0.8214285714285714
accuracy :  0.9250936329588015
['엠씨넥스'] : 097520
roc_auc :  0.9096242584047463
f1_score ;  0.6754966887417219
accuracy :  0.8164794007490637
['CJ제일제당'] : 097950
roc_auc :  0.9566502463054187
f1_score ;  0.676056338028169
accuracy :  0.9138576779026217
['비상교육'] : 100220
roc_auc :  0.954876615746181
f1_score ;  0.7142857142857142
accuracy :  0.9250936329588015
['진양홀딩스'] : 100250
roc_auc :  0.9746015936254979
f1_score ;  0.6896551724137931
accuracy :  0.9662921348314607
['동성케미컬'] : 102260
roc_auc :  0.9888888888888889
f1_score ;  0.7777777777777777
accuracy :  0.9550561797752809
['쌍방울'] : 102280
roc_auc :  0.9140394088669951
f1_score ;  0.6071428571428571
accuracy :  0.9176029962546817
['일진전기'] : 103590
roc_auc :  0.9729947138588829
f1_score ;  0.7631578947368421
accuracy :  0.9325842696629213
['한세실업'] : 105630
roc_auc :  0.9352847822253572
f1_score ;  0.7706422018348623
accuracy :  0.9063670411985019
['우진'] : 105840
roc_auc :  

roc_auc :  0.9718571618213195
f1_score ;  0.8606060606060606
accuracy :  0.9138576779026217
['SK디앤디'] : 210980
roc_auc :  0.9589484589484589
f1_score ;  0.7575757575757576
accuracy :  0.9400749063670412
['한솔제지'] : 213500
roc_auc :  0.9732838589981447
f1_score ;  0.6818181818181818
accuracy :  0.947565543071161
['이노션'] : 214320
roc_auc :  0.9598393574297188
f1_score ;  0.5945945945945946
accuracy :  0.9438202247191011
['금호에이치티'] : 214330
roc_auc :  0.9742857142857142
f1_score ;  0.860759493670886
accuracy :  0.9588014981273408
['토니모리'] : 214420
roc_auc :  0.9547547547547547
f1_score ;  0.7529411764705882
accuracy :  0.9213483146067416
['잇츠한불'] : 226320
roc_auc :  0.9552995391705069
f1_score ;  0.7032967032967035
accuracy :  0.898876404494382
['엘브이엠씨'] : 900140
roc_auc :  0.9653621232568601
f1_score ;  0.7945205479452054
accuracy :  0.9438202247191011


In [211]:
xg_accuracy_10 = []
xg_accuracy_f1_10 = []

for file in files[:]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/30일/train/{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name,':',file[:6])
    
    y = train_df['10per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    xg = XGBClassifier()

    model = [rfc, xg, lbgm]
    model_name = ['RFC','XG', 'LBGM']
    
    
    xg.fit(x_train, y_train)
    pred = xg.predict(x_test)   
    prob = xg.predict_proba(x_test)

    with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_10per/마지막_test/XGB/{file[:6]}.pickle','wb') as fw:
        pickle.dump(m, fw)

#     print(classification_report(y_test, pred))
#     print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('f1_score ; ', f1_score(y_test, pred))
    print('accuracy : ', accuracy_score(y_test, pred))
    xg_accuracy_10.append(roc_auc_score(y_test, prob[:,1]))
    xg_accuracy_f1_10.append(f1_score(y_test, pred))
#     print('-'*100)
        
        
    
    print('='*100)

['KR모터스'] : 000040
[22:58:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9688926940639269
f1_score ;  0.787878787878788
accuracy :  0.9213483146067416
['경방'] : 000050
[22:58:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9217249657064472
f1_score ;  0.5499999999999999
accuracy :  0.9325842696629213
['하이트진로'] : 000080
[22:58:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default ev

roc_auc :  0.9486051301254447
f1_score ;  0.8409090909090908
accuracy :  0.947565543071161
['삼일제약'] : 000520
[22:58:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9385880077369438
f1_score ;  0.6818181818181819
accuracy :  0.8951310861423221
['CS홀딩스'] : 000590
[22:58:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9825970548862115
f1_score ;  0.7894736842105262
accuracy :  0.9700374531835206
['동아쏘시오홀딩스'] : 000640
[22:58:10] WARNING: C:/Users/Administrator/wor

roc_auc :  0.9509607351712615
f1_score ;  0.7058823529411765
accuracy :  0.8876404494382022
['CJ'] : 001040
[22:58:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9552406657669815
f1_score ;  0.7058823529411765
accuracy :  0.9250936329588015
['JW중외제약'] : 001060
[22:58:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9524150268336316
f1_score ;  0.7956989247311829
accuracy :  0.9288389513108615
['대한방직'] : 001070
[22:58:12] WARNING: C:/Users/Administrator/workspa

roc_auc :  0.9522523557802804
f1_score ;  0.71875
accuracy :  0.9325842696629213
['디아이동일'] : 001530
[22:58:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9887747336377473
f1_score ;  0.9032258064516129
accuracy :  0.9662921348314607
['제일연마'] : 001560
[22:58:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9036947304663839
f1_score ;  0.5217391304347826
accuracy :  0.9588014981273408
['금양'] : 001570
[22:58:14] WARNING: C:/Users/Administrator/workspace/xgboost-w

roc_auc :  0.9345475910693302
f1_score ;  0.7297297297297297
accuracy :  0.9250936329588015
['한일철강'] : 002220
[22:58:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9524247299471387
f1_score ;  0.7567567567567567
accuracy :  0.9325842696629213
['고려제강'] : 002240
[22:58:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9542737597130903
f1_score ;  0.6250000000000001
accuracy :  0.9325842696629213
['아세아제지'] : 002310
[22:58:16] WARNING: C:/Users/Administrator/worksp

...

roc_auc :  0.9790809327846366
f1_score ;  0.7999999999999999
accuracy :  0.9662921348314607
['에이블씨엔씨'] : 078520
[22:59:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9611677073063861
f1_score ;  0.8723404255319148
accuracy :  0.9478260869565217
['CJ CGV'] : 079160
[22:59:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9526214526214526
f1_score ;  0.6666666666666666
accuracy :  0.9213483146067416
['현대리바트'] : 079430
[22:59:01] WARNING: C:/Users/Administrator/wo

roc_auc :  0.9720744680851063
f1_score ;  0.75
accuracy :  0.947565543071161
['메타랩스'] : 090370
[22:59:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9807862843629914
f1_score ;  0.8920863309352517
accuracy :  0.9438202247191011
['아모레퍼시픽'] : 090430
[22:59:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9500894454382827
f1_score ;  0.7391304347826088
accuracy :  0.9101123595505618
['세원이앤씨'] : 091090
[22:59:03] WARNING: C:/Users/Administrator/workspace/xgboost-w

roc_auc :  0.9595769682726205
f1_score ;  0.6944444444444444
accuracy :  0.9176029962546817
['진양홀딩스'] : 100250
[22:59:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9746015936254979
f1_score ;  0.7407407407407406
accuracy :  0.9737827715355806
['동성케미컬'] : 102260
[22:59:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9928973277074543
f1_score ;  0.8571428571428571
accuracy :  0.9700374531835206
['쌍방울'] : 102280
[22:59:06] WARNING: C:/Users/Administrator/worksp

f1_score ;  0.875
accuracy :  0.9700374531835206
['대성산업'] : 128820
[22:59:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9630898513251455
f1_score ;  0.8192771084337349
accuracy :  0.8876404494382022
['한미약품'] : 128940
[22:59:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.955852534562212
f1_score ;  0.6881720430107526
accuracy :  0.8913857677902621
['인터지스'] : 129260
[22:59:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.c

roc_auc :  0.9736842105263158
f1_score ;  0.8829787234042554
accuracy :  0.9176029962546817
['아세아시멘트'] : 183190
[22:59:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9767621145374449
f1_score ;  0.7692307692307692
accuracy :  0.9325842696629213
['쿠쿠홀딩스'] : 192400
[22:59:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9190860893713709
f1_score ;  0.6363636363636364
accuracy :  0.9101123595505618
['코스맥스'] : 192820
[22:59:10] WARNING: C:/Users/Administrator/work

In [212]:
print('rfc : ',sum(rfc_accuracy)/len(rfc_accuracy))
print('xg : ',sum(xg_accuracy)/len(xg_accuracy))
print('lbgm : ',sum(lbgm_accuracy)/len(lbgm_accuracy))

rfc :  0.9514731458990663
xg :  0.936343853270578
lbgm :  0.9379878229724077


## 30일 15per

In [213]:
rfc_accuracy_15 = []
rfc_accuracy_f1_15 = []

for file in files[:]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/30일/train/{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name,':',file[:6])

    y = train_df['15per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    rfc = RandomForestClassifier()

    rfc.fit(x_train, y_train)
    pred = rfc.predict(x_test)   
    prob = rfc.predict_proba(x_test)

    with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_15per/마지막_test/Randomforest/{file[:6]}.pickle','wb') as fw:
        pickle.dump(m, fw)


#     print(classification_report(y_test, pred))
#     print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('f1_score ; ', f1_score(y_test, pred))
    print('accuracy : ', accuracy_score(y_test, pred))
    rfc_accuracy_15.append(roc_auc_score(y_test, prob[:,1]))
    rfc_accuracy_f1_15.append(f1_score(y_test, pred))
#     print('-'*100)



    print('='*100)
        
#     except:
#         print(stock_name,':',file[:6])

['KR모터스'] : 000040
roc_auc :  0.9775661375661375
f1_score ;  0.7848101265822786
accuracy :  0.9363295880149812
['경방'] : 000050
roc_auc :  0.9974160206718345
f1_score ;  0.7777777777777778
accuracy :  0.9850187265917603
['하이트진로'] : 000080
roc_auc :  0.9836032388663968
f1_score ;  0.8205128205128205
accuracy :  0.9737827715355806
['유한양행'] : 000100
roc_auc :  0.9804048234280792
f1_score ;  0.6250000000000001
accuracy :  0.9775280898876404
['CJ대한통운'] : 000120
roc_auc :  0.9957386363636364
f1_score ;  0.761904761904762
accuracy :  0.9812734082397003
['두산'] : 000150
roc_auc :  0.9396551724137931
f1_score ;  0.7741935483870968
accuracy :  0.947565543071161
['성창기업지주'] : 000180
roc_auc :  0.9884588068181819
f1_score ;  0.7500000000000001
accuracy :  0.9775280898876404
['DL'] : 000210
roc_auc :  0.9323828345567476
f1_score ;  0.64
accuracy :  0.9662921348314607
['일동홀딩스'] : 000230
roc_auc :  0.9912478825522303
f1_score ;  0.6956521739130435
accuracy :  0.9737827715355806
['한국앤컴퍼니'] : 000240
roc_a

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [214]:
xg_accuracy_15 = []
xg_accuracy_f1_15 = []

for file in files[:]:
    train_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/30일/train/{file}',index_col=0)
    stock_name = str(name_df[name_df.stock_code == file[:6]]['corp_name'].values)
    print(stock_name,':',file[:6])
    
    y = train_df['15per']
    x = train_df.dropna().drop(['date','등락률','수익률','quarter','5per','10per','15per'], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    xg = XGBClassifier()

    model = [rfc, xg, lbgm]
    model_name = ['RFC','XG', 'LBGM']
    
    
    xg.fit(x_train, y_train)
    pred = xg.predict(x_test)   
    prob = xg.predict_proba(x_test)

    with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/30일_15per/마지막_test/XGB/{file[:6]}.pickle','wb') as fw:
        pickle.dump(m, fw)

#     print(classification_report(y_test, pred))
#     print('-'*50)
    print('roc_auc : ',roc_auc_score(y_test, prob[:,1]))
    print('f1_score ; ', f1_score(y_test, pred))
    print('accuracy : ', accuracy_score(y_test, pred))
    xg_accuracy_15.append(roc_auc_score(y_test, prob[:,1]))
    xg_accuracy_f1_15.append(f1_score(y_test, pred))
#     print('-'*100)
        
        
    
    print('='*100)

['KR모터스'] : 000040
[23:05:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9728042328042329
f1_score ;  0.8
accuracy :  0.9363295880149812
['경방'] : 000050
[23:05:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc :  0.9926787252368647
f1_score ;  0.7777777777777778
accuracy :  0.9850187265917603
['하이트진로'] : 000080
[23:05:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metri

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [219]:
train_df['15per'].value_counts()

0    1321
1      11
Name: 15per, dtype: int64

In [ ]:
### fold

In [143]:
df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_통합_data/{file}',index_col=0)
df

,date,시가,고가,저가,종가,거래량,거래대금,등락률,종목명,시가총액,...,부채총계,자본총계,매출액,영업이익,법인세차감전 순이익,당기순이익,수익률,5per,10per,15per
0,2016-05-02,10350,10400,9800,9800,605826,6064879720,-4.39,900140,469121178400,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-17.0,0,0,0
1,2016-05-03,9870,9950,9520,9620,676470,6547925390,-1.84,900140,460504666960,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-17.0,0,0,0
2,2016-05-04,9610,9650,9410,9430,522646,4954491670,-1.98,900140,451409460440,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-11.0,0,0,0
3,2016-05-09,9450,9630,9370,9480,307304,2925183970,0.53,900140,453802935840,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-10.0,0,0,0
4,2016-05-10,9480,9630,9340,9440,375936,3547769450,-0.42,900140,451888155520,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-11.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,2022-02-09,3290,3345,3260,3335,225672,745088895,1.68,900140,331683415495,...,192946247.0,365001684.0,143756198.0,-4652956.0,-9211434.0,-9304838.0,0.0,0,0,0
1420,2022-02-10,3335,3390,3250,3270,155480,509499320,-1.95,900140,325218821190,...,192946247.0,365001684.0,143756198.0,-4652956.0,-9211434.0,-9304838.0,2.0,0,0,0
1421,2022-02-11,3270,3280,3200,3265,181605,588128895,-0.15,900140,324721544705,...,192946247.0,365001684.0,143756198.0,-4652956.0,-9211434.0,-9304838.0,3.0,0,0,0
1422,2022-02-14,3245,3245,3060,3125,219802,686248160,-4.29,900140,310797803125,...,192946247.0,365001684.0,143756198.0,-4652956.0,-9211434.0,-9304838.0,8.0,1,0,0


In [192]:
df['2016-05' < df.date < '2017-03']


,date,시가,고가,저가,종가,거래량,거래대금,등락률,종목명,시가총액,...,부채총계,자본총계,매출액,영업이익,법인세차감전 순이익,당기순이익,수익률,5per,10per,15per
0,2016-05-02,10350,10400,9800,9800,605826,6064879720,-4.39,900140,469121178400,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-17.0,0,0,0
1,2016-05-03,9870,9950,9520,9620,676470,6547925390,-1.84,900140,460504666960,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-17.0,0,0,0
2,2016-05-04,9610,9650,9410,9430,522646,4954491670,-1.98,900140,451409460440,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-11.0,0,0,0
3,2016-05-09,9450,9630,9370,9480,307304,2925183970,0.53,900140,453802935840,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-10.0,0,0,0
4,2016-05-10,9480,9630,9340,9440,375936,3547769450,-0.42,900140,451888155520,...,218843232.0,331902273.0,88111636.0,10192059.0,6770401.0,6770401.0,-11.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,2017-02-22,7990,8160,7990,8110,207381,1675613120,2.40,900140,388221709880,...,208553458.0,362485427.0,-5006209.0,37840273.0,30305771.0,30247156.0,-10.0,0,0,0
202,2017-02-23,8190,8190,7980,7990,190771,1531924610,-1.48,900140,382477368920,...,208553458.0,362485427.0,-5006209.0,37840273.0,30305771.0,30247156.0,-8.0,0,0,0
203,2017-02-24,8010,8080,7840,7860,227421,1801781700,-1.63,900140,376254332880,...,208553458.0,362485427.0,-5006209.0,37840273.0,30305771.0,30247156.0,-8.0,0,0,0
204,2017-02-27,7880,7880,7630,7700,276890,2138164690,-2.04,900140,368595211600,...,208553458.0,362485427.0,-5006209.0,37840273.0,30305771.0,30247156.0,-6.0,0,0,0
